In [17]:
from datetime import datetime
import numpy as np
import pandas as pd

# Define the custom dtype for the structured array
dtype = np.dtype([
    ('datetime', 'datetime64[m]'),
    ('open', 'f4'),
    ('high', 'f4'),
    ('low', 'f4'),
    ('close', 'f4')
])

# Read the CSV file
df_eurusd = pd.read_csv('./Data/eurusd_2021to2024.csv')

# Convert the datetime column to datetime64
df_eurusd['datetime'] = pd.to_datetime(df_eurusd['datetime'])

# Create the structured array
data_eurusd = np.zeros(len(df_eurusd), dtype=dtype)

# Fill the structured array with data from the dataframe
data_eurusd['datetime'] = df_eurusd['datetime'].values.astype('datetime64[m]')
data_eurusd['open'] = df_eurusd['open'].values
data_eurusd['high'] = df_eurusd['high'].values
data_eurusd['low'] = df_eurusd['low'].values
data_eurusd['close'] = df_eurusd['close'].values


In [18]:
# Read the CSV file
df_btcusd = pd.read_csv('./Data/btcusd_2022to2024.csv')

# Convert the datetime column to datetime64
df_btcusd['datetime'] = pd.to_datetime(df_btcusd['datetime'])

# Create the structured array
data_btcusd = np.zeros(len(df_btcusd), dtype=dtype)

# Fill the structured array with data from the dataframe
data_btcusd['datetime'] = df_btcusd['datetime'].values.astype('datetime64[m]')
data_btcusd['open'] = df_btcusd['open'].values
data_btcusd['high'] = df_btcusd['high'].values
data_btcusd['low'] = df_btcusd['low'].values
data_btcusd['close'] = df_btcusd['close'].values

In [19]:
"""
Klasse für Positionen, soll bei jedem kauf geöffnet werden und bei jedem verkauf geschlossen
"""

class Position:
    def __init__(self, ticker, quantity, price, leverage, margin, type) -> None:
        self.ticker = ticker
        self.quantity = quantity
        self.price_whenopened = price
        self.price = price
        self.total_whenopened = self.quantity * self.price
        self.total = self.quantity * self.price
        self.leverage = leverage
        self.margin = margin
        self.pnl = 0
        self.type = type

    def add_quantity(self, quantity_to_add, margin_to_add, price):
        self.quantity = self.quantity + quantity_to_add
        self.total_whenopened += quantity_to_add * price
        self.total += quantity_to_add * price
        self.margin += margin_to_add
        self.leverage = self.total_whenopened / self.margin
    
    def remove_quantity(self, quantity_to_remove):
        percentage = quantity_to_remove / self.quantity
        self.quantity = self.quantity - quantity_to_remove
        self.total_whenopened = self.quantity * self.price_whenopened
        self.total = self.quantity * self.price
        self.margin = self.margin * (1-percentage)
        self.pnl = self.total - self.total_whenopened
        
    def update_price(self, new_price):
        self.price = new_price
        self.total = self.quantity * self.price
        self.pnl = self.total - self.total_whenopened
        
    
    def close(self):
        self = None

In [20]:
from typing import List

"""
Klasse die das Kapital und die Positionen eines Users speichert und verwaltet
"""

class User:
    def __init__(self, name, cash):
        self.name = name
        self.cash = cash                # Starting cash
        self.capital_invested = 0
        self.capital = self.cash + self.capital_invested
        self.positions: List[Position] = []
        
    def buy_stock(self, ticker, quantity, price, leverage, type):
        if quantity*price <= self.cash:
            margin = (quantity * price) / leverage
            self.capital_invested += margin
            self.cash -= margin
            self.capital = self.cash + self.capital_invested
            self.positions.append(Position(ticker, quantity, price, leverage, margin, type))
            print(f"{self.name} bought Stock {ticker} with Leverage: {leverage}")
        
        elif ticker in [position.ticker for position in self.positions]:
            margin = (quantity * price) / leverage
            self.capital_invested += margin
            self.cash -= margin
            self.capital = self.cash + self.capital_invested
            #get the position that already exists with the same ticker
            position = next((position for position in self.positions if position.ticker == ticker), None) 
            position.add_quantity(quantity, margin, price)
        else:
            print("Not enough cash")

    def sell_stock(self, postion:Position, quantity):
        if postion.quantity == quantity:
            self.capital_invested -= postion.margin + postion.pnl
            self.cash += postion.margin + postion.pnl
            self.capital = self.cash + self.capital_invested
            self.positions.remove(postion)
            postion.close()
        elif postion.quantity > quantity:
            percentage = quantity / postion.quantity
            self.capital_invested -=( postion.margin + postion.pnl) * percentage
            self.cash += (postion.margin + postion.pnl) * percentage
            self.capital = self.cash + self.capital_invested
            postion.remove_quantity(quantity)
        else:
            print("Not enough stock to sell")
            
    def get_cash(self):
        return self.cash

    def get_positions(self):
        return self.positions

In [44]:
class GameManager:
    """
    datetime must be in Year-Month-Day Minutes:Hours
    """
    def __init__(self, start_date, tickers) -> None:
        self.start_date = start_date
        self.tickers = tickers
        
    def get_stock_price(self, ticker, timedate):
        if ticker in self.tickers:
            if ticker == 'EUR/USD':
                timedate = np.datetime64(timedate, 'm')
                mask = (data_eurusd['datetime'] == timedate)
                return data_eurusd[mask]
        else:
            print("Not supported ticker symbol")
    
    def get_stock_prices(self, ticker, start_date, end_date):
        if ticker in self.tickers:
            if ticker == 'EUR/USD':
                start_date = np.datetime64(start_date, 'm')
                end_date = np.datetime64(end_date, 'm')
                # Create a boolean mask
                mask = (data_eurusd['datetime'] >= start_date) & (data_eurusd['datetime'] <= end_date)
                return data_eurusd[mask]
        else:
            print("Not supported ticker symbol")

# Tests

In [40]:
Test = GameManager("2022-06-06", 2)

In [41]:
print(Test.get_stock_price("EUR/USD", "2022-08-09 07:22"))

[('2022-08-09T07:22', 1.0193, 1.0193, 1.0193, 1.0193)]


In [43]:
print(Test.get_stock_prices("EUR/USD", "2022-08-09 07:20", "2022-08-10 00:00"))

[('2022-08-09T07:22', 1.0193 , 1.0193 , 1.0193 , 1.0193 )
 ('2022-08-09T07:23', 1.0192 , 1.0195 , 1.0192 , 1.0195 )
 ('2022-08-09T07:24', 1.0195 , 1.0195 , 1.0193 , 1.0193 )
 ('2022-08-09T07:27', 1.0195 , 1.0196 , 1.0194 , 1.0194 )
 ('2022-08-09T07:28', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:29', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:30', 1.0194 , 1.0195 , 1.0194 , 1.0195 )
 ('2022-08-09T07:31', 1.0194 , 1.0195 , 1.0194 , 1.0195 )
 ('2022-08-09T07:32', 1.0195 , 1.0195 , 1.01945, 1.01945)
 ('2022-08-09T07:33', 1.0194 , 1.0195 , 1.0193 , 1.0193 )
 ('2022-08-09T07:34', 1.0195 , 1.0195 , 1.0194 , 1.0194 )
 ('2022-08-09T07:35', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:36', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:37', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:38', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:39', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T07:40', 1.0194 , 1.0194 , 1.0194 , 1.0194 )
 ('2022-08-09T